<a href="https://colab.research.google.com/github/amityu/AmitMichael/blob/master/notebookfe9cc2d688.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 12.3 MB/s eta 0:00:00


In [52]:
import pandas as pd
from pytorch_lightning.utilities.types import EVAL_DATALOADERS
#import YARCN
import torch
import torch.nn as nn
import random
import numpy as np
import torchmetrics
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl

from torch.utils.data import random_split
CHECK_PATH =r'D:\amityu\rec_data' +'/cp/'
dnn_path = 'C:/Users/amityu/red_data_c/dnn/'
DATA_PATH = 'C:/Users/amityu/red_data_c/'
DATA_PATHD = 'D:/amityu/rec_data/'

class RCM(pl.LightningModule):

    def __init__(self,max_user_id= 544873, max_item_id=51270, max_source=93150, max_campaign = 19000, dim=64,
                 user_id_dim = 1, item_id_dim=16, source_dim =16, campaign_dim = 12,
                 placement_id_dim = 8, syndicator_dim = 8, region_dim =8,
                 placement_max =1405, syndicator_max = 2350, region_max = 1035):
        super().__init__()
        user_numerical_columns = ['page_view_start_time', 'user_recs', 'user_clicks', 'user_target_recs']
        item_numerical_columns = [    'empiric_calibrated_recs','empiric_clicks']
        user_oh_cat_columns = list(pd.read_csv('/content/drive/MyDrive/reco2/user_oh_col')['column'])
        item_oh_cat_columns = list(pd.read_csv('/content/drive/MyDrive/reco2/item_oh_col')['column'])

        self.save_hyperparameters()
        self.placement_id = nn.Embedding(num_embeddings=placement_max, embedding_dim=placement_id_dim)
        self.syndicator = nn.Embedding(num_embeddings=syndicator_max, embedding_dim=syndicator_dim)
        self.region = nn.Embedding(num_embeddings=region_max, embedding_dim=region_dim)
        self.target_id = nn.Embedding(num_embeddings=max_item_id, embedding_dim=item_id_dim)

        #self.user_id = nn.Embedding(num_embeddings=max_user_id, embedding_dim=user_id_dim)
        self.source = nn.Embedding(num_embeddings=max_source, embedding_dim=source_dim)

        self.campaign = nn.Embedding(num_embeddings=max_campaign, embedding_dim=campaign_dim)

        self.fc_user = nn.Linear(in_features=len(user_numerical_columns + user_oh_cat_columns), out_features=dim)
        #self.fc_user_full = nn.Linear(in_features=user_id_dim + region_dim + dim, out_features=dim)
        self.fc_user_full = nn.Linear(in_features= region_dim + dim, out_features=dim)
        #self.fc_user_full = nn.Linear(in_features=dim, out_features=dim)

        self.fc_item = nn.Linear(in_features=len(item_oh_cat_columns + item_numerical_columns) , out_features=dim)
        self.fc_item_full = nn.Linear(in_features= dim + campaign_dim + source_dim + item_id_dim + syndicator_dim +
                                                   placement_id_dim, out_features=dim)
        #self.fm = nn.Linear(in_features = dim*2, out_features =dim)
        self.deep = nn.Linear(in_features=dim*2, out_features=dim*2)
        self.validation_auc = torchmetrics.AUROC(task='binary')
        self.output = nn.Linear(2*dim, out_features=1)
        self.dim = dim

    def forward(self, x):
        (user, region, item, target_id, source,campaign, syndicator, placement_id) = x
        # Pass through embedding layers
        relu = nn.ReLU()
        user_embedded = relu(self.fc_user(user))
        #user_id_embedded = self.user_id(user_id)
        region_embedded = self.region(region)
        #user_embedded_full = relu(self.fc_user_full(torch.cat([user_id_embedded, region_embedded, user_embedded], dim=1)))
        user_embedded_full = relu(self.fc_user_full(torch.cat([region_embedded, user_embedded], dim=1)))


        item_embedded = relu(self.fc_item(item))
        item_id_embedded = self.target_id(target_id)
        source_embedded = self.source(source)
        campaign_embedded = self.campaign(campaign)
        syndicator_embedded = self.syndicator(syndicator)
        placement__embedded = self.placement_id(placement_id)
        item_embedded_full = relu(self.fc_item_full(torch.cat([item_embedded, item_id_embedded, source_embedded, campaign_embedded, syndicator_embedded, placement__embedded], dim=1)))

        #fac_machine_vector = relu(self.fm(torch.cat([user_embedded_full, item_embedded_full], dim=1)))
        deep_vector = relu(self.deep(torch.cat([user_embedded_full, item_embedded_full], dim=1)))
        pred = self.output(deep_vector)
        return pred

    def training_step(self, batch, batch_idx):
        x, labels = batch

        predicted_labels = self(x)
        loss = nn.BCEWithLogitsLoss()(predicted_labels.view(-1,1), labels.view(-1,1).float())
        return loss

    def validation_step(self, batch, batch_idx):
        #(user, user_id,region, item, target_id, source,campaign, syndicator, placement_id), labels = batch
        x, labels = batch

        predicted_logits = self(x)
        loss = nn.BCEWithLogitsLoss()(predicted_logits.view(-1,1), labels.view(-1,1).float())
        self.log('val_loss', loss, prog_bar=True)
        preds = torch.sigmoid(predicted_logits)
        self.validation_auc.update(preds.squeeze(), labels)
        self.log('val_auc', self.validation_auc, prog_bar=True)

    #def on_validation_epoch_end(self, outputs):
    #    self.log('val_auc_epoch', self.validation_auc.compute(), prog_bar=True)
    #    self.validation_auc.reset()
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr = 0.001)
        return opt




In [53]:
class FMds2(Dataset):
    def __init__(self,ds_df):

        user_numerical_columns = ['page_view_start_time', 'user_recs', 'user_clicks', 'user_target_recs']
        item_numerical_columns = [    'empiric_calibrated_recs','empiric_clicks']
        user_oh_cat_columns = list(pd.read_csv('/content/drive/MyDrive/reco2/user_oh_col')['column'])
        item_oh_cat_columns = list(pd.read_csv('/content/drive/MyDrive/reco2/item_oh_col')['column'])
        print(user_oh_cat_columns)
        self.user = ds_df[user_oh_cat_columns + user_numerical_columns].astype(np.float32)
        self.item = ds_df[item_oh_cat_columns + item_numerical_columns].astype(np.float32)
        self.target_id = ds_df['target_id_hash'].astype(int)
        self.source = ds_df['source_id_hash'].astype(int)
        self.campaign = ds_df['campaign_id_hash'].astype(int)
        #self.user_id = ds_df['user_id_hash']
        self.region = ds_df['region'].astype(int)
        self.syndicator = ds_df['syndicator_id_hash'].astype(int)
        self.placement_id = ds_df['placement_id_hash'].astype(int)
        self.labels = ds_df['is_click'].astype(int)
    def __getitem__(self, index):
        return (torch.tensor(self.user.iloc[index]), torch.tensor(self.region.iloc[index]),
                torch.tensor(self.item.iloc[index]), torch.tensor(self.target_id.iloc[index]), torch.tensor(self.source.iloc[index]),
                torch.tensor(self.campaign.iloc[index]), torch.tensor(self.syndicator.iloc[index]), torch.tensor(self.placement_id.iloc[index]),
                ), torch.tensor(self.labels.iloc[index])
    def __len__(self):
        return len (self.user)




In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
df = pd.read_csv('/content/drive/MyDrive/reco2/train_small_few_cat.csv)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/reco2/train_df_encoded.csv'

In [ ]:
ds = FMds2(df)

#%%
#del df
#%%
train_size = int(0.8 * len(ds))  # 80% for training
val_size = len(ds) - train_size  # Remaining 20% for validation
train_dataset, val_dataset = random_split(ds, [train_size, val_size])
del ds
#%%
#%%
model = RCM()
#model =RCM.load_from_checkpoint('/content/drive/MyDrive/reco2/cp3/FM-epoch=02-val_loss=0.48.ckpt')
trainer = pl.Trainer(num_sanity_val_steps=0, max_epochs=150,fast_dev_run=False, callbacks=[pl.callbacks.ModelCheckpoint
                                                                (monitor='val_auc',dirpath='/content/drive/MyDrive/reco2/cp2',filename='FM-{epoch:02d}-{val_loss:.2f}',save_top_k=3,mode='min')])
trainer.fit(model, train_dataloaders=DataLoader(train_dataset, batch_size=512, num_workers=2), val_dataloaders=DataLoader(val_dataset, batch_size=512, num_workers=2))#,ckpt_path='/content/drive/MyDrive/reco2/cp2/FM-epoch=19-val_loss=0.81.ckpt')
print('done')

['browser_platform_DESK', 'browser_platform_PHON', 'browser_platform_TBLT', 'browser_platform_TV', 'os_family_0', 'os_family_1', 'os_family_2', 'os_family_3', 'os_family_4', 'os_family_5', 'os_family_6', 'country_code_+100', 'country_code_+200', 'country_code_+300', 'country_code_+530', 'country_code_-300', 'country_code_-400', 'country_code_-500', 'country_code_-600', 'country_code_-700', 'country_code_24', 'country_code_250', 'country_code_301/240', 'country_code_330/234', 'country_code_361', 'country_code_405', 'country_code_417', 'country_code_559', 'country_code_606', 'country_code_630/331', 'country_code_631/934', 'country_code_641', 'country_code_68', 'country_code_712', 'country_code_719', 'country_code_732/848', 'country_code_76', 'country_code_77', 'country_code_787/939', 'country_code_806', 'country_code_83', 'country_code_90', 'country_code_905', 'country_code_912', 'country_code_925', 'country_code_95', 'country_code_99', 'country_code_AE', 'country_code_AF', 'country_code

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory /content/drive/MyDrive/reco2/cp2 exists and is not empty.
INFO:pytorch_lightning.callbacks.model_summary:
   | Name           | Type        | Params
------------------------------------------------
0  | placement_id   | Embedding   | 11.2 K
1  | syndicator     | Embedding   | 18.8 K
2  | region         | Embedding   | 8.3 K 
3  | target_id      | Embedding   | 820 K 
4  | source         | Embedding   | 1.5 M 
5  | campaign       | Embedding   | 228 K 
6  | fc_user        | Linear      | 21.6 K
7  | fc_user_full   | Linear      | 4.7 K 
8  | fc_item        | Lin

Training: |          | 0/? [00:00<?, ?it/s]

In [47]:
model =RCM.load_from_checkpoint('/content/drive/MyDrive/reco2/cp2/FM-epoch=02-val_loss=0.47.ckpt')
test_df = pd.read_csv('/content/drive/MyDrive/reco2/test_df_encoded.csv')

model.eval()
with torch.no_grad():  # Disable gradient computation for inference
        user_numerical_columns = ['page_view_start_time', 'user_recs', 'user_clicks', 'user_target_recs']
        item_numerical_columns = [    'empiric_calibrated_recs','empiric_clicks']
        user_oh_cat_columns = list(pd.read_csv('/content/drive/MyDrive/reco2/user_oh_col')['column'])
        item_oh_cat_columns = list(pd.read_csv('/content/drive/MyDrive/reco2/item_oh_col')['column'])
        user = test_df[user_oh_cat_columns + user_numerical_columns].astype(np.float32)
        item = test_df[item_oh_cat_columns + item_numerical_columns].astype(np.float32)
        target_id = test_df['target_id_hash'].astype(int)
        source = test_df['source_id_hash'].astype(int)
        campaign = test_df['campaign_id_hash'].astype(int)
        #self.user_id = ds_df['user_id_hash']
        region = test_df['region'].astype(int)
        syndicator = test_df['syndicator_id_hash'].astype(int)
        placement_id = test_df['placement_id_hash'].astype(int)
        x = (torch.tensor(user.values), torch.tensor(region.values), torch.tensor(item.values), torch.tensor(target_id.values), torch.tensor(source.values), torch.tensor(campaign.values), torch.tensor(syndicator.values), torch.tensor(placement_id.values))
        predictions = torch.sigmoid(model(x))
        p_df = pd.DataFrame(predictions, columns=['is_click'])
        p_df.to_csv('/content/drive/MyDrive/reco2/predictions_colab.csv', index=False)

In [49]:
import pandas as pd
p_df = pd.read_csv('/content/drive/MyDrive/reco2/predictions_colab.csv')
#p_df['Predicted'] = p_df['is_click']
p_df.reset_index(inplace=True)
p_df.columns =['Id','Predicted']
p_df.to_csv('/content/drive/MyDrive/reco2/my_submission_colab_loss47.csv', index=False)